# Creating New Phenotype Dataset and Merging Genetic Data with Accelerometer Cohort

### Sources:

> Walmsley R, Naushad J, Klempir O, Busby B and Doherty A. rap_wearables (2022), URL: https://github.com/OxWearables/rap_wearables.
>
> DNA Nexus Tutorial: https://www.youtube.com/watch?v=jodNjrYF8po&t=1056s


## Creating All Ancestry Phenotype Cohort (Pheno Cohort)

In this section, I created a cohort containing people of all ancestries and merged the genetic and phenotypic data. I also formally create the coronary artery disease variable here. The goal of creating a cohort *distinct from the accelerometer cohort* was to allow me to evaluate the performance of the PRS in a population similar to that used in Tamlander *et al.* 


I started by creating this new cohort “Processed Cohort ALL ANCESTRIES” in the Cohort Browser. I applied the typical genotype quality control procedures *with the exception of kinship restrictions*, which I handled later. These criteria included:

* Genotype calls IS NOT NULL (Excluding individuals who were not genotyped)
* Reason lost to follow-up IS NOT participant has withdrawn consent for future linkage
* sex chromosome aneuploidy IS NOT Yes
* Outliers for heterozygosity or missing rate IS NOT Yes 

This yields a dataset containing 486,432 individuals.



## Spark JL Code for Pheno Cohort, Hospital Inpatient Data, Death Data

I then use Spark Jupyter Lab to create a CSV of the phenotype cohort that contains ALL possible codes for CAD and some of the covariates (more added later) and saved as “FULLCADCOHORT.csv.” **This is also where I add the stipulation that genetic sex = biological sex.**

This creates a few datasets: phenotype dataset, hospital episode, and cause and time of death datasets. Hospital inpatient data and death data are acquired exactly the same way as in "Creating Accelerometer Cohort" and so the code is skipped.

In [ ]:
import pyspark
import dxpy
import dxdata
import pandas as pd
import os

from pyspark.sql.functions import when, concat_ws
from re import sub


In [ ]:
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)


In [ ]:

# Discover dispensed database name and dataset id
dispensed_database_name = dxpy.find_one_data_object(classname = "database",
    name = "app*", folder = "/",
    name_mode = "glob",
    describe = True)["describe"]["name"]

dispensed_dataset_id = dxpy.find_one_data_object(typename = "Dataset",
 name = "app*.dataset",
 folder = "/",
 name_mode = "glob")["id"]


In [ ]:

dataset = dxdata.load_dataset(id = dispensed_dataset_id)


In [ ]:

participant = dataset["participant"]


In [ ]:

cohort = dxdata.load_cohort("Processed Cohort ALL ANCESTRIES")


field_names = ["eid", "p22001", "p22006", "p31", "p191", "p21022", "p52", "p34", "p738_i0", "p738_i1", "p738_i2", "p738_i3",
              "p6138_i0", "p6138_i1", "p6138_i2", "p6138_i3", "p20160_i0", "p20160_i1", "p20160_i2",
              "p20160_i3", "p20116_i0", "p20116_i1", "p20116_i2", "p20116_i3", "p3894_i0", "p3894_i1",
              "p3894_i2", "p3894_i3", "p3627_i0", "p3627_i1", "p3627_i2", "p3627_i3", "p4056_i0", "p4056_i1",
              "p4056_i2", "p4056_i3", "p21001_i0", "p21001_i1", "p21001_i2", "p21001_i3",
              "p23104_i0","p23104_i1", "p23104_i2", "p23104_i3", "p48_i0", "p48_i1", "p48_i2", "p48_i3",
              "p49_i0", "p49_i1", "p49_i2", "p49_i3", "p40000_i0", "p40000_i1", "p40007_i0", "p40007_i1",
              "p40001_i0", "p40001_i1", "p42000", "p42006", "p131296", "p131297", "p131298", "p131299",
              "p131300", "p131301", "p131302", "p131303", "p131304", "p131305", "p131306", "p131307",
              "p131342", "p131343", "p131354", "p131355", "p131360", "p131361", "p131362", "p131363",
              "p131364", "p131365", "p131366", "p131367", "p131368", "p131369", "p131378", "p131379",
              "p53_i0", "p53_i1", "p53_i2", "p53_i3", "p21003_i0", "p21003_i1", "p21003_i2", "p21003_i3",
      "p54_i0", "p54_i1", "p54_i2", "p54_i3", "p41272", "p41282_a0", "p41282_a1", "p41282_a2", "p41282_a3",
              "p41282_a4","p41282_a5", "p41282_a6", "p41282_a7", "p41282_a8", "p41282_a9", "p41282_a10",
              "p41282_a11", "p41282_a12", "p41282_a13", "p41282_a14",
              "p41282_a15","p41282_a16", "p41282_a17", "p41282_a18", "p41282_a19", "p41282_a20", "p41282_a21",
              "p41282_a22", "p41282_a23", "p41282_a24", "p41282_a25",
              "p41282_a26","p41282_a27", "p41282_a28", "p41282_a29", "p41282_a30", "p41282_a31", "p41282_a32",
              "p41282_a33", "p41282_a34", "p41282_a35", "p41282_a36",
              "p41282_a37","p41282_a38", "p41282_a39", "p41282_a40", "p41282_a41", "p41282_a42", "p41282_a43",
              "p41282_a44", "p41282_a45", "p41282_a46", "p41282_a47",
              "p41282_a48","p41282_a49", "p41282_a50", "p41282_a51", "p41282_a52", "p41282_a53", "p41282_a54",
              "p41282_a55", "p41282_a56", "p41282_a57", "p41282_a58",
              "p41282_a59","p41282_a60", "p41282_a61", "p41282_a62", "p41282_a63", "p41282_a64", "p41282_a65",
              "p41282_a66", "p41282_a67", "p41282_a68", "p41282_a69",
              "p41282_a70","p41282_a71", "p41282_a72", "p41282_a73", "p41282_a74", "p41282_a75", "p41282_a76",
              "p41282_a77", "p41282_a78", "p41282_a79", "p41282_a80",
              "p41282_a81","p41282_a82", "p41282_a83", "p41282_a84", "p41282_a85", "p41282_a86", "p41282_a87",
              "p41282_a88", "p41282_a89", "p41282_a90", "p41282_a91",
              "p41282_a92","p41282_a93", "p41282_a94", "p41282_a95", "p41282_a96", "p41282_a97", "p41282_a98",
              "p41282_a99", "p41282_a100", "p41282_a101", "p41282_a102",
              "p41282_a103","p41282_a104", "p41282_a105", "p41282_a106", "p41282_a107", "p41282_a108", "p41282_a109",
              "p41282_a110", "p41282_a111", "p41282_a112", "p41282_a113",
              "p41282_a114","p41282_a115", "p41282_a116", "p41282_a117", "p41282_a118", "p41282_a119", "p41282_a120",
              "p41282_a121", "p41282_a122", "p41282_a123", "p41283_a0", "p41283_a1", "p41283_a2", "p41283_a3",
              "p41283_a4","p41283_a5", "p41283_a6", "p41283_a7", "p41283_a8", "p41283_a9", "p41283_a10",
              "p41283_a11", "p41283_a12", "p41283_a13", "p41283_a14",
              "p41283_a15","p22021","p41273"]


In [ ]:

df = participant.retrieve_fields(names = field_names,
 filter_sql = cohort.sql,
 coding_values = "replace",
 engine = dxdata.connect())


In [ ]:
# Making sure genetic and biological sex match
df = df[(df['p31'] == df['p22001'])


In [ ]:

df_pandas = df.toPandas()
df_pandas.head()

df_pandas.to_csv("FULLCADCOHORT.csv")


In [ ]:
%%bash
dx upload FULLCADCOHORT.csv



## Creating Coronary Artery Disease Cases & Merging Genotype Data & Pheno Cohort

I next load all these datasets into “PROCESSING FOR MARS PGS TEST.R” and use this code to create the CAD case definition, death, region, status, and time variables and apply censoring by region. I apply the CAD case definitions in “CAD Definition in Tamlander et al,” which makes use of ICD-10 and -9 codes for deaths, hospital inpatient episodes, and OPCS-3 and OPCS-4 codes for surgery. This R file ultimately yields “FULLPHENODATAALLANCESTRIES.csv.” *NOTE: Kinship restrictions still not done yet.*

Relevant codes for the creation of coronary artery disease include:

**Death Register** <br />

*ICD-10 Codes:*

- I20 to I25
- I46
- R96


**Hospital Inpatient Episodes** <br />

*ICD-10 Codes:*
- I20.0
- I21-I22

*ICD-9 Codes:*
- 410
- 4110


**Surgical Procedures** <br />

*OPCS-4 Codes:*
- K40 to K46
- K49
- K501
- K75

*OPCS-3 Codes:*
- 3043


In [ ]:
# Bash kernel
dx download "FULLCADCOHORT.csv" # Pheno Cohort
dx download "dathes.csv" # Hospital inpatient records
dx download "datdeath.csv" # Death register recorsd
dx download "datdeathcause.csv" # Causes of death

In [ ]:
# R kernel

# Loading required packages
install.packages('stringr')
library(stringr)

install.packages('data.table')
library(data.table)

install.packages('plyr')
library(plyr)

install.packages("lubridate")
library(lubridate)


UKBDF <- read.csv("FULLCADCOHORT.csv")


# First subsetting to only eids and surgeries to keep relevant ones and dates
UKBDFOPS <- UKBDF[ , c(2, 102:242, 244)]


In [ ]:

# Using p41272 - Operative procedures with OPCS4 Codes to restrict to relevant surgeries
UKBDFOPS$RELEVANTOP4 <- grepl('K40|K41|K42|K43|K44|K45|K46|K49|K501|K75', UKBDFOPS$p41272)
# Creates T/F variable for whether operation contained is in our criteria


# Using p41273 - Operative procedures with OPCS3 Codes to restrict to relevant surgeries
UKBDFOPS$RELEVANTOP3 <- grepl('3043', UKBDFOPS$p41273)
# Creates T/F variable for whether operation contained is in our criteria
# There ARE no relevant operations from OPCS-3! So can do normally


In [ ]:

# Creating relevant operation variable to factor all of them in
UKBDFOPS$RELEVANTOP <- ifelse(UKBDFOPS$RELEVANTOP4 == TRUE | UKBDFOPS$RELEVANTOP3 == TRUE, TRUE, FALSE)


In [ ]:
# Count number of commas BEFORE operation and have that determine the array chosen
# Variable contains all operations a person has ever had separated by commas
# So can count the commas to determine which of the 123 possible arrays to choose

# Only extract string BEFORE operation - subsequent operations aren't relevant (first diagnosis/surgery)
# This restricts to ONLY operations BEFORE these relevant operations
UKBDFOPS$p41272 <- sub("K40.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K41.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K42.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K43.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K44.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K45.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K46.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K49.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K501.*", "", UKBDFOPS$p41272)
UKBDFOPS$p41272 <- sub("K75.*", "", UKBDFOPS$p41272)

UKBDFOPS$p41273 <- sub("3043.*", "", UKBDFOPS$p41273)

In [ ]:
# ONLY record as non-missing if individual has record of correct operation and count commas to get correct array
# OPCS-3
UKBDFOPS$V3 <- ifelse(UKBDFOPS$RELEVANTOP3 == TRUE, str_count(UKBDFOPS$p41273, ','), NA)
# This code works for selecting array because FIRST array is actually a0 NOT a1

# OPCS-4
UKBDFOPS$V4 <- ifelse(UKBDFOPS$RELEVANTOP4 == TRUE, str_count(UKBDFOPS$p41272, ','), NA)

In [ ]:
# Using number of commas found BEFORE operation to choose array variable
# First creating column name variables
UKBDFOPS$V5 <- paste("p41283_a", UKBDFOPS$V3, sep = "")
UKBDFOPS$V6 <- paste("p41282_a", UKBDFOPS$V4, sep = "")


# Choosing column based on these column name variables
UKBDFOPS$MATCH <- match(UKBDFOPS$V5, names(UKBDFOPS))
UKBDFOPS$MATCH2 <- match(UKBDFOPS$V6, names(UKBDFOPS))


In [ ]:
# MATCH - works if matching EITHER OPCS-3 OR OCPS-4 code
UKBDFOPS$MATCH3 <- ifelse(is.na(UKBDFOPS$MATCH) == FALSE | is.na(UKBDFOPS$MATCH2) == FALSE, TRUE, FALSE)


# Only keeping where NO NAs - individuals with relevant operations
UKBDFOPS <- subset(UKBDFOPS, RELEVANTOP == TRUE)


In [ ]:
# Creating empty object within which to store operation dates
OPDATE <- matrix(NA, nrow = nrow(UKBDFOPS), ncol = 1)

# Create as separate data frame and then join back after creating OPDATE vector
# Using ONLY MATCH2 here to match because NO OPCS-3 operations matched
# This creates a vector of the relevant operation dates
for(i in 1:nrow(UKBDFOPS)){
  OPDATE[i, ] <- UKBDFOPS[i, UKBDFOPS$MATCH2[i]]
  
}

# Adding OPDATE vector as variable to broader operation dataset
UKBDFOPS$OperationDate <- OPDATE


In [ ]:
# Find indices for EID and Operation Date
summary(match("eid", colnames(UKBDFOPS)))
summary(match("OperationDate", colnames(UKBDFOPS)))
# 1 and 154

# Drop all variables except EID and Operation Date - merging w/ full UKB Dataset
colnames(UKBDFOPS)
dim(UKBDFOPS)
UKBDFOPS <- UKBDFOPS[ , c(1, 154)]

# Remerge with UKBDF
UKBDFMerge <- merge(UKBDF, UKBDFOPS, by = "eid", all = T)


colnames(UKBDFMerge)
dim(UKBDFMerge)
# 21,718 operations
# 486,432 individuals x 245 variables

In [ ]:
# Creating REGION variable
UKBDFMerge$REGION <- ifelse(UKBDFMerge$p54_i0 == "Cardiff" | UKBDFMerge$p54_i0 == "Swansea" | UKBDFMerge$p54_i0 == "Wrexham", "Wales",
                            ifelse(UKBDFMerge$p54_i0 == "Edinburgh" | UKBDFMerge$p54_i0 == "Glasgow", "Scotland", "England"))

summary(as.factor(UKBDFMerge$REGION))
# England 431306
# Scotland 34833
# Wales 20293


In [ ]:

# Renaming the last two - MAKE SURE THESE ARE STILL IN THOSE SAME SPOTS
colnames(UKBDFMerge)[245] <- "p999999" #OperationDate
colnames(UKBDFMerge)[246] <- "p9999990" # REGION

# Renaming EID AS WELL
colnames(UKBDFMerge)[1] <- "p9999999999" # EID

# Removing X from variable list so all are formatted the same
UKBDFMerge <- UKBDFMerge[-2]


In [ ]:
# Creating vector of variable names
varnames <- c("p9999999999","p9999990","p22001", "p22006", "p31", "p191", "p21022", "p52", "p34", "p738_i0", "p738_i1", "p738_i2", "p738_i3",
              "p6138_i0", "p6138_i1", "p6138_i2", "p6138_i3", "p20160_i0", "p20160_i1", "p20160_i2",
              "p20160_i3", "p20116_i0", "p20116_i1", "p20116_i2", "p20116_i3", "p3894_i0", "p3894_i1",
              "p3894_i2", "p3894_i3", "p3627_i0", "p3627_i1", "p3627_i2", "p3627_i3", "p4056_i0", "p4056_i1",
              "p4056_i2", "p4056_i3", "p21001_i0", "p21001_i1", "p21001_i2", "p21001_i3",
              "p23104_i0","p23104_i1", "p23104_i2", "p23104_i3", "p48_i0", "p48_i1", "p48_i2", "p48_i3",
              "p49_i0", "p49_i1", "p49_i2", "p49_i3", "p40000_i0", "p40000_i1", "p40007_i0", "p40007_i1",
              "p40001_i0", "p40001_i1", "p42000", "p42006", "p131296", "p131297", "p131298", "p131299",
              "p131300", "p131301", "p131302", "p131303", "p131304", "p131305", "p131306", "p131307",
              "p131342", "p131343", "p131354", "p131355", "p131360", "p131361", "p131362", "p131363",
              "p131364", "p131365", "p131366", "p131367", "p131368", "p131369", "p131378", "p131379",
              "p53_i0", "p53_i1", "p53_i2", "p53_i3", "p21003_i0", "p21003_i1", "p21003_i2", "p21003_i3","p9999990","p999999",
              "p54_i0", "p54_i1", "p54_i2", "p54_i3", "p41272", "p41273", "p41282_a0", "p41282_a1", "p41282_a2", "p41282_a3",
              "p41282_a4","p41282_a5", "p41282_a6", "p41282_a7", "p41282_a8", "p41282_a9", "p41282_a10",
              "p41282_a11", "p41282_a12", "p41282_a13", "p41282_a14",
              "p41282_a15","p41282_a16", "p41282_a17", "p41282_a18", "p41282_a19", "p41282_a20", "p41282_a21",
              "p41282_a22", "p41282_a23", "p41282_a24", "p41282_a25",
              "p41282_a26","p41282_a27", "p41282_a28", "p41282_a29", "p41282_a30", "p41282_a31", "p41282_a32",
              "p41282_a33", "p41282_a34", "p41282_a35", "p41282_a36",
              "p41282_a37","p41282_a38", "p41282_a39", "p41282_a40", "p41282_a41", "p41282_a42", "p41282_a43",
              "p41282_a44", "p41282_a45", "p41282_a46", "p41282_a47",
              "p41282_a48","p41282_a49", "p41282_a50", "p41282_a51", "p41282_a52", "p41282_a53", "p41282_a54",
              "p41282_a55", "p41282_a56", "p41282_a57", "p41282_a58",
              "p41282_a59","p41282_a60", "p41282_a61", "p41282_a62", "p41282_a63", "p41282_a64", "p41282_a65",
              "p41282_a66", "p41282_a67", "p41282_a68", "p41282_a69",
              "p41282_a70","p41282_a71", "p41282_a72", "p41282_a73", "p41282_a74", "p41282_a75", "p41282_a76",
              "p41282_a77", "p41282_a78", "p41282_a79", "p41282_a80",
              "p41282_a81","p41282_a82", "p41282_a83", "p41282_a84", "p41282_a85", "p41282_a86", "p41282_a87",
              "p41282_a88", "p41282_a89", "p41282_a90", "p41282_a91",
              "p41282_a92","p41282_a93", "p41282_a94", "p41282_a95", "p41282_a96", "p41282_a97", "p41282_a98",
              "p41282_a99", "p41282_a100", "p41282_a101", "p41282_a102",
              "p41282_a103","p41282_a104", "p41282_a105", "p41282_a106", "p41282_a107", "p41282_a108", "p41282_a109",
              "p41282_a110", "p41282_a111", "p41282_a112", "p41282_a113",
              "p41282_a114","p41282_a115", "p41282_a116", "p41282_a117", "p41282_a118", "p41282_a119", "p41282_a120",
              "p41282_a121", "p41282_a122", "p41282_a123", "p41283_a0", "p41283_a1", "p41283_a2", "p41283_a3",
              "p41283_a4","p41283_a5", "p41283_a6", "p41283_a7", "p41283_a8", "p41283_a9", "p41283_a10",
              "p41283_a11", "p41283_a12", "p41283_a13", "p41283_a14",
              "p41283_a15","p22021")



In [ ]:
# Remove p to sort numerically then add back in
# Getting ordered numerically
varnames <- gsub("p", "", varnames)
varnames <- varnames[order(as.numeric(varnames))]
varnames <- paste("p", varnames, sep = "")

In [ ]:
# Doing the same for UKB Dataset as above

# First remove p
colnames(UKBDFMerge) <- gsub("p", "", colnames(UKBDFMerge))

# Now order numerically - WHAT ABOUT NON-NUMERIC PARTS????
UKBDFMerge <- UKBDFMerge[ , order(as.numeric(colnames(UKBDFMerge)))]

# Adding p back in
colnames(UKBDFMerge) <- paste("p", colnames(UKBDFMerge), sep = "")

In [ ]:
# After getting order of varnames and UKBDFMerge to match

# Renaming fields to be interpretable - after organizing numerically
colnames(UKBDFMerge) <- c("Biological Sex", "Year of Birth", "Month of Birth", "Date Lost to Follow Up",
                          "Age at Recruitment", "Genetic Sex", "Genetic Ethnic Grouping", "Genetic Kinship", "OPCs4 Procedure", "OPCs3 Procedure", "Date of MI", "Date of Stroke", "Date of I20",
                          "Source of I20", "Date of I21", "Source of I21", "Date of I22", "Source of I22", "Date of I23", "Source of I23",
                          "Date of I24", "Source of I24", "Date of I25", "Source of I25", "Date of I44", "Source of I44",
                          "Date of I50", "Source of I50", "Date of I60", "Source of I60", "Date of I61", "Source of I61",
                          "Date of I62", "Source of I62", "Date of I63", "Source of I63", "Date of I64", "Source of I64",
                          "Date of I69", "Source of I69", "OperationDate", "REGION", "ID","HH Income Inst 0", "HH Income Inst 1", "HH Income Inst 2", "HH Income Inst 3",
                          "EA Inst 0", "EA Inst 1", "EA Inst 2", "EA Inst 3", "EverSmoke Inst 0", "EverSmoke Inst 1", "EverSmoke Inst 2",
                          "EverSmoke Inst 3", "SmokStat Inst 0", "SmokStat Inst 1", "SmokStat Inst 2", "SmokStat Inst 3", "Age Heart Attack Inst 0",
                          "Age Heart Attack Inst 1", "Age Heart Attack Inst 2", "Age Heart Attack Inst 3", "Age Angina Inst 0", "Age Angina Inst 1",
                          "Age Angina Inst 2", "Age Angina Inst 3", "Age Stroke Inst 0", "Age Stroke Inst 1", "Age Stroke Inst 2", "Age Stroke Inst 3",
                          "Measured BMI Inst 0", "Measured BMI Inst 1", "Measured BMI Inst 2", "Measured BMI Inst 3", "Imped BMI Inst 0", "Imped BMI Inst 1",
                          "Imped BMI Inst 2", "Imped BMI Inst 3", "WC Inst 0", "WC Inst 1", "WC Inst 2", "WC Inst 3", "HC Inst 0", "HC Inst 1", "HC Inst 2", "HC Inst 3",
                          "Date of Death Inst 0", "Date of Death Inst 1", "Age at Death Inst 0", "Age at Death Inst 1", "Primary Cause of Death Inst 0", "Primary Cause of Death Inst 1",
                          "Date Attending Assess Center Inst 0", "Date Attending Assess Center Inst 1", "Date Attending Assess Center Inst 2", "Date Attending Assess Center Inst 3",
                          "Age Attending Assess Center Inst 0", "Age Attending Assess Center Inst 1", "Age Attending Assess Center Inst 2", "Age Attending Assess Center Inst 3")



In [ ]:
# Remove extraneous REGION, OPERATION variables (and extra X)
UKBDFMerge <- UKBDFMerge[ , -c(102:245)]


In [ ]:
# NOW reading in hospital and death data
dat_hes <- fread("dathes.csv", data.table = FALSE)
dat_death <- fread("datdeath.csv", data.table = FALSE)
dat_death_cause <- fread("datdeathcause.csv", data.table = FALSE)

In [ ]:
# Restrict hospital inpatient data to subset of variables
cols_dat_hes <- c("eid","dnx_hesin_id", "dnx_hesin_diag_id", 
                  "dateepiimp",  "ins_index", "arr_index", "level",
                  "diag_icd9", "diag_icd9_nb", "diag_icd10", "diag_icd10_nb")



dat_hes <- dat_hes[, cols_dat_hes]


In [ ]:
# Converting hospital data date to Date data type
dat_hes$date_hes <- as.Date(dat_hes$dateepiimp, format = "%Y-%m-%d")


# Death data
dat_death$date_death <- as.Date(dat_death$date_of_death, format = "%Y-%m-%d")


dat_death <- dat_death[dat_death$ins_index == 0, ]

In [ ]:
# The lists of ICD codes we will consider for hospital inpatient episodes
icd10_codes <- "I20.0|I21|I22"
icd9_codes <- "^410|^4110"


# Restrict the hospital data frame to occurrences of relevant codes and only EID, date, hospital episodes, ICD diagnoses
dat_hes_rel <- dat_hes[grepl(icd10_codes, dat_hes$diag_icd10) | grepl(icd9_codes, dat_hes$diag_icd9), c("eid", "date_hes", "diag_icd10", "diag_icd9") ]


# Find first occurrence
dat_hes_first_cad <- aggregate(dat_hes_rel$date_hes, list(dat_hes_rel$eid), min)
colnames(dat_hes_first_cad) <- c("eid", "date_hes_first_cad")


In [ ]:
# Creating correctly written ID variable for merging
UKBDFMerge$eid <- UKBDFMerge$ID


# Merge full UKB dataset and hospital inpatient records of CAD (left join)
dat <- merge(UKBDFMerge, 
             dat_hes_first_cad,
             by = "eid",
             all.x = TRUE,
             suffixes = c("", "dup")
)



In [ ]:
# Merging full dataset and date of death
dat <-
  merge(
    dat,
    dat_death[, c("eid", "date_death")],
    by = "eid",
    all.x = TRUE,
    suffixes = c("", "dup") # This makes it safe if we accidentally run it twice - we won't rename the columns
  )


In [ ]:
# ICD-10 codes from Death Register
ids_death_cad <- dat_death_cause$eid[grepl("I2[0-5]|I46|R96", dat_death_cause$cause_icd10)]
# Vector of all EIDs who died from CAD

length(ids_death_cad)
# 8787 cases of CAD-specific death


In [ ]:
# NOW have inds w/ causes of death as CAD (counts as EVENT)

# Restrict to CAD-specific death data frame
dat_death_cad <- dat_death[dat_death$eid %in% ids_death_cad, ]

# Getting dates associated with CAD-related deaths
dat_death_cad$date_of_death_cad <- dat_death_cad$date_of_death

# Restricting this dataset to only EID and date of death from CAD
dat_death_cadsub <- dat_death_cad[c(2,8)]

In [ ]:

# Merging CAD-specific cause of death to larger dataset
dat <-
  merge(
    dat,
    dat_death_cadsub,
    by = "eid",
    all.x = TRUE,
    suffixes = c("", "dup") 
  )


In [ ]:
# Making date of birth Variable
# ONLY take last two variables of Year
dat$Year <- sprintf('%02d', dat$`Year of Birth` %% 100)
dat$Month <- match(dat$`Month of Birth`, month.name)

# Convert into DOB - arbitrarily chose first of the month
dat$DOB <- as.Date(paste(dat$`Year of Birth`, dat$Month, 01, sep = "-"))

In [ ]:
# Finding indices for relevant CAD-related operations, hospital inpatient episodes, and deaths
summary(match("OperationDate", colnames(dat)))
summary(match("date_hes_first_cad", colnames(dat)))
summary(match("date_of_death_cad", colnames(dat)))
# 14, 75, 77

# SUPPRESS WARNINGS FIRST
options(warn=-1)

# First CAD date based on earliest date between operations, hospital inpatient episodes, and deaths
dat$CADDate <- apply(dat[ , c(14, 75, 77)], 1, FUN = min, na.rm = TRUE)
# 47,238 PREVALENT CAD
# 229,003 CONTROLS

In [ ]:

# Create Status variable denoting whether someone did or did not have a CVD event
dat$Status <- ifelse(is.na(dat$CADDate) == FALSE, 1, 0)

summary(as.factor(dat$Status))
# 30,350 cases
# 456,082 controls

In [ ]:
# Converting CAD date to date type
dat$CADDate <- as.Date(dat$CADDate)

# Make sure these dates attending assessment center are coded as dates
dat$`Date Attending Assess Center Inst 0` <- as.Date(dat$`Date Attending Assess Center Inst 0`)
dat$`Date Attending Assess Center Inst 1` <- as.Date(dat$`Date Attending Assess Center Inst 1`)
dat$`Date Attending Assess Center Inst 2` <- as.Date(dat$`Date Attending Assess Center Inst 2`)
dat$`Date Attending Assess Center Inst 3` <- as.Date(dat$`Date Attending Assess Center Inst 3`)

In [ ]:
# Creating death variable that is EXPLICITLY excluding CAD
dat$date_of_death_nocad <- ifelse(is.na(dat$date_of_death_cad) == TRUE, dat$date_of_death, NA)

In [ ]:
# Write this result to CSV
FULLPHENODATA <- write.csv(dat, "FULLPHENODATAALLANCESTRIES.csv")

In [ ]:
#  Bash Kernel
dx upload FULLPHENODATAALLANCESTRIES.csv

## Adding Genetic Principal Components to Dataset

I had to add in the genetic principal components, which follows a similar process as before. This yielded PCsforCAD.csv.

In [ ]:
import pyspark
import dxpy
import dxdata
import pandas as pd
import os

from pyspark.sql.functions import when, concat_ws
from re import sub

In [ ]:
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

In [ ]:
dispensed_database_name = dxpy.find_one_data_object(classname = "database",
    name = "app*", folder = "/",
    name_mode = "glob",
    describe = True)["describe"]["name"]

dispensed_dataset_id = dxpy.find_one_data_object(typename = "Dataset",
 name = "app*.dataset",
 folder = "/",
 name_mode = "glob")["id"]

In [ ]:
dataset = dxdata.load_dataset(id = dispensed_dataset_id)

In [ ]:
participant = dataset["participant"]

In [ ]:
cohort = dxdata.load_cohort("Processed Cohort ALL ANCESTRIES")


field_names = ["eid", "p22009_a1", "p22009_a2", "p22009_a3", "p22009_a4", "p22009_a5",
      "p22009_a6", "p22009_a7", "p22009_a8", "p22009_a9", "p22009_a10",
      "p22009_a11", "p22009_a12", "p22009_a13", "p22009_a14", "p22009_a15",
      "p22009_a16", "p22009_a17", "p22009_a18", "p22009_a19", "p22009_a20",
      "p22009_a21", "p22009_a22", "p22009_a23", "p22009_a24", "p22009_a25",
      "p22009_a26", "p22009_a27", "p22009_a28", "p22009_a29", "p22009_a30",
      "p22009_a31", "p22009_a32", "p22009_a33", "p22009_a34", "p22009_a35",
      "p22009_a36", "p22009_a37", "p22009_a38", "p22009_a39", "p22009_a40"]

In [ ]:
df = participant.retrieve_fields(names = field_names,
 filter_sql = cohort.sql,
 coding_values = "replace",
 engine = dxdata.connect())

In [ ]:
df_pandas = df.toPandas()
df_pandas.head()

In [ ]:
df_pandas.to_csv("PCsforCAD.csv")

In [ ]:
# Bash kernel
dx upload PCsforCAD.csv